In [ ]:
13%%time
#SPINNEYS DATA SCRAPPING
import re
import urllib.request,urllib.parse,urllib.error
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
metascore_w = re.compile('metascore_w.*')
Rating= re.compile('cert_rating.*')

url='https://www.metacritic.com/browse/movies/score/metascore/all'
base='https://www.metacritic.com'
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,}
Movies = pd.DataFrame(columns=['Number','Title','Ratingstatus','Realeasedate','metascore','userscore','movielink'])
request=urllib.request.Request(url,None,headers)
response=urllib.request.urlopen(request)
soup=BeautifulSoup(response,'html.parser')
i=0
while (request):
  i=i+1
  try:
    Hirerchicalcont=soup.body.findAll('td',{"clamp-summary-wrap"})
    for item in Hirerchicalcont:
      Number=item.findAll('span',{"title numbered"})
      Titles=item.findAll('a',{"title"})
      ratingstatus=item.findAll('span',{"class":Rating})
      realeasedate=item.findAll('div',{"clamp-details"})
      metascores=item.findAll('div',{"clamp-metascore"})
      userscores=item.findAll('div',{"clamp-userscore"})
      if ratingstatus is not None and len(ratingstatus) > 0:
        rating = ratingstatus[0].text
      else:
        rating='null'
        if Titles[0]['href']=='Citizen Kane':
          Titles[0]['href']='Citizen Kane-1941'

      Movies = Movies.append({"Number":Number[0].text.strip()[:-1],"Title":Titles[0].text,"Ratingstatus": rating[2:], "Realeasedate":realeasedate[0].span.text,"metascore":
               metascores[0].find('div',{"class":metascore_w}).text,"userscore":userscores[0].find('div',{"class":metascore_w}).text,"movielink":base+Titles[0]['href']},ignore_index=True)
  except IndexError:
    continue
  try
      request=urllib.request.Request(url+'?page='+ str(i),None,headers)
      print(url+'?page='+ str(i))
      response=urllib.request.urlopen(request)
      soup=BeautifulSoup(response,'html.parser')
      time.sleep(3)
  except IndexError:
    break



In [ ]:
Movies.to_csv('Movies.csv',index=False)

In [2]:
import numpy as np
import pandas as pd

!git clone https://github.com/AhmadIbrahim-AI/data-scraping.git
# if folder is already cloned and you want to update it 
# %cd /content/data-scraping
# ! git pull 

Movies_Data=pd.read_csv('/content/data-scraping/Movies of all time.csv')
Movies_Data.head(5)

fatal: destination path 'data-scraping' already exists and is not an empty directory.


,Number,Title,Ratingstatus,Realeasedate,metascore,userscore,movielink
0,1,Citizen Kane,Approved,"September 4, 1941",100,8.4,https://www.metacritic.com/movie/citizen-kane-...
1,2,The Godfather,R,"March 11, 1972",100,9.2,https://www.metacritic.com/movie/the-godfather
2,3,Rear Window,TV-G,"September 1, 1954",100,8.8,https://www.metacritic.com/movie/rear-window
3,4,Casablanca,TV-PG,"January 23, 1943",100,8.9,https://www.metacritic.com/movie/casablanca
4,5,Boyhood,R,"July 11, 2014",100,7.5,https://www.metacritic.com/movie/boyhood


In [3]:
print('there are {} Movies in metacritic website.'.format(Movies_Data.shape[0]))

there are 14451 Movies in metacritic website.


In [4]:
Movies_Data=Movies_Data[0:2500]
Movies_Data.head(1000)

,Number,Title,Ratingstatus,Realeasedate,metascore,userscore,movielink
0,1,Citizen Kane,Approved,"September 4, 1941",100,8.4,https://www.metacritic.com/movie/citizen-kane-...
1,2,The Godfather,R,"March 11, 1972",100,9.2,https://www.metacritic.com/movie/the-godfather
2,3,Rear Window,TV-G,"September 1, 1954",100,8.8,https://www.metacritic.com/movie/rear-window
3,4,Casablanca,TV-PG,"January 23, 1943",100,8.9,https://www.metacritic.com/movie/casablanca
4,5,Boyhood,R,"July 11, 2014",100,7.5,https://www.metacritic.com/movie/boyhood
...,...,...,...,...,...,...,...
995,996,Cunningham,Not Rated,"December 13, 2019",83,tbd,https://www.metacritic.com/movie/cunningham
996,997,The Flat,Not Rated,"October 12, 2012",83,8.2,https://www.metacritic.com/movie/the-flat
997,998,"Bloody Nose, Empty Pockets",Not Rated,"July 8, 2020",83,7.3,https://www.metacritic.com/movie/bloody-nose-e...
998,999,Blind,Not Rated,"September 4, 2015",83,7.6,https://www.metacritic.com/movie/blind


In [ ]:
#Extracting Reviews for each Movie
%%time
#SPINNEYS DATA SCRAPPING
import re
import urllib.request,urllib.parse,urllib.error
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import re
Reviews = pd.DataFrame(columns=['Number','Title','User Reviews','USER SCORE'])

for i, j in Movies_Data.iterrows():
    # if j[0]==2100 or j[0]==2200 or j[0]==2300  or j[0]==2400  :
    #   print('enter')
    #   time.sleep(20)
    f=0
    url=j[6] + '/user-reviews'
    print(url)
    metascore_w = re.compile('metascore_w user large movie.*')

    user_agent = 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36 '
    headers={'User-Agent':user_agent,}
    request=urllib.request.Request(url,None,headers)
    response=urllib.request.urlopen(request)
    soup=BeautifulSoup(response,'lxml')
    try:
      if soup.body.find('div',{"pad_top1 pad_btm2"}) != '' and len(soup.body.find('div',{"pad_top1 pad_btm2"}))>0:
        if soup.body.find('div',{"pad_top1 pad_btm2"}).text=='No reviews yet.':
          continue
    except:
        Hirerchicalcont=soup.body.findAll('div',{"review pad_top1"})
        if len(Hirerchicalcont)==0:
           try:
            url=j[6]+"-"+j[3][-4:] + '/user-reviews'
            request=urllib.request.Request(url,None,headers)
            response=urllib.request.urlopen(request)
            soup=BeautifulSoup(response,'lxml')
            Hirerchicalcont=soup.body.findAll('div',{"review pad_top1"})
           except:
             continue
        while (request):
          f=f+1
          for  item in Hirerchicalcont:
            try:
              if item.find('span',{"blurb blurb_expanded"})!= '' and len(item.find('span',{"blurb blurb_expanded"}))>0:
                userreview= re.sub(r'[\t\r\n]', '', item.find('span',{"blurb blurb_expanded"}).text )
                userscore=item.find('div',{'class':metascore_w}).get_text()
                Reviews = Reviews.append({"Number":j[0],"Title":j[1],"User Reviews": userreview, "USER SCORE":userscore},ignore_index=True)
            except:
              try:
                userreview=item.find('div',{"summary"}).span.text
                userscore=item.find('div',{'class':metascore_w}).text
                #re.sub(r'[\t\r\n]', '', userreview) to remove New Line </br>
                Reviews = Reviews.append({"Number":j[0],"Title":j[1],"User Reviews": re.sub(r'[\t\r\n]', '', userreview), "USER SCORE":userscore},ignore_index=True)
                continue
              except:
                continue
                
          try:
            print(str(j[0])+" " + url+'?page='+ str(f))
            request=urllib.request.Request(url+'?page='+ str(f),None,headers)
            response=urllib.request.urlopen(request)
            soup=BeautifulSoup(response,'lxml')
            Hirerchicalcont=soup.body.findAll('div',{"review pad_top1"})
          
            try:
              if soup.body.find('div',{"pad_top1 pad_btm2"}) != '' and len(soup.body.find('div',{"pad_top1 pad_btm2"}))>0:
                if soup.body.find('div',{"pad_top1 pad_btm2"}).text=='No reviews yet.':
                  break
            except:
              continue
          except IndexError:
            break

In [ ]:
Reviews.to_csv('Reviews.csv',index=False)